# Simulating a 2D Stabilized Supralinear Network

In [ ]:
# Installation
using Revise
using LinearAlgebra,Statistics,StatsBase,Distributions
using Plots,NamedColors ; theme(:default)
using FFTW
using ProgressMeter
using Random
Random.seed!(0)
using NamedColors
col_exc = colorant"dark blue"
col_inh = colorant"dark red"

In [ ]:
# Initializing variables for a 1 E, 1 I network
v_rest = -70
alpha = 0.3
n = 2
w = [1.25 -0.65
     1.2 -0.5]
tau = [20.0, 10.0]*1E-3   #time constants : [ tau_E, tau_I ]
time_step = 1*1E-3
tau_noise = 50*1E-3

In [ ]:
function onedmat(x::Real)
  return cat(x;dims=2)
end

In [ ]:
# Input-Output function : r(V) = alpha*( V - V_rest )^n
function rate_powerlaw(v::Real, v_rest::Real, n::Real, alpha::Real)
    diff = v - v_rest
    diff = (diff < 0) ? 0 : diff
    return alpha*(diff^n)
end

function plot_powerlaw()
    count = 81
    v_arr = zeros(count)
    v_arr[1] = -90
    for i in 2:count
        v_arr[i] = v_arr[i-1] +1
    end
    rates = rate_powerlaw.(v_arr, v_rest, n, alpha)
    plot(v_arr, rates, xlabel="Voltage (mV)", ylabel="Rate (Hz)", color="black", legend=false, fmt=:png)
end

plot_powerlaw()

In [ ]:
function spiking(probability)
    return rand(Float64) < probability
end

In [ ]:
# modelling noise as a Multivariate Ornstein-Uhlenbeck process
function dnoise!(noise::Vector{Float64}, wiener::Vector{Float64}, cov_noise::Matrix{Float64})
    wiener = wiener + sqrt(time_step)*[rand(Normal(0,0.5)),rand(Normal(0,0.5))]
    dn = - noise .* time_step + sqrt(2*tau_noise*cov_noise)*wiener
    return dn ./ tau_noise
end

In [ ]:
# Euler Method
function dv_ssn(v::Real, v_rest::Real, h_i::Real, noise_i::Real, tau_i::Real, w_arr::Vector{Float64}, rate_arr::Vector{Float64})
    num_neurons = size(w_arr, 1)
    dv = v_rest - v + h_i + noise_i
    for i in 1:num_neurons
        dv += w_arr[i]*rate_arr[i]
    end
    return dv*time_step/tau_i
end

In [ ]:
function simulate_ssn!(h, num_steps, t_arr, v_excite, v_inhibit, v_rest, rate, spike_trainE, spike_trainI, cov_noise)
    
    v_excite[1] = v_rest
    v_inhibit[1] = v_rest
    rate[1,:] = [rate_powerlaw(v_excite[1], v_rest, n, alpha), rate_powerlaw(v_inhibit[1], v_rest, n, alpha)]
    noise = zeros(2)
    wiener = zeros(2)
    @showprogress 1.0 "Running SSN..." for i in 2:num_steps
        t_arr[i] = t_arr[i-1] + time_step
        noise[:] = noise[:] .+ dnoise!(noise, wiener, cov_noise)
        v_excite[i] = v_excite[i-1] + dv_ssn(v_excite[i-1], v_rest, h, noise[1], tau[1], [w[1,1], w[1,2]], rate[i-1,:])
        v_inhibit[i] = v_inhibit[i-1] + dv_ssn(v_inhibit[i-1], v_rest, h, noise[2], tau[2], [w[2,1], w[2,2]], rate[i-1,:])
        rate[i,:] = [rate_powerlaw(v_excite[i], v_rest, n, alpha), rate_powerlaw(v_inhibit[i], v_rest, n, alpha)]
        
        spike_probability = rate[i,:] .* (time_step)
        if spiking(spike_probability[1])
            push!(spike_trainE, t_arr[i])
        end
        if spiking(spike_probability[2])
            push!(spike_trainI, t_arr[i])
        end
    end
    
end

Running the SSN Simulation for t_max time, with a fixed external input h, with no external noise

In [ ]:
function single_h_no_noise_ssn_simulation()
    h = 5.0
    t_max = 1
    num_steps = Int(ceil(t_max/time_step))
    t_arr = zeros(num_steps)
    v_excite = zeros(num_steps)
    v_inhibit = zeros(num_steps)
    spike_trainE = Vector{Float64}()
    spike_trainI = Vector{Float64}()
    rate = zeros(num_steps, 2)
    cov_noise = [0.0 0.0
                0.0 0.0]    # setting cov_noise to zero to remove noise from the system
    simulate_ssn!(h, num_steps, t_arr, v_excite, v_inhibit, v_rest, rate, spike_trainE, spike_trainI, cov_noise)
    return t_arr, v_excite, v_inhibit, rate, spike_trainE, spike_trainI
end

In [ ]:
t_arr, v_excite, v_inhibit, rate, spike_trainE, spike_trainI = single_h_no_noise_ssn_simulation()
println([size(spike_trainE), size(spike_trainI)])

In [ ]:
length = size(t_arr,1)
plt = plot(xlabel="time (s)", ylabel="Voltage (mV)", fmt = :png, legend=:bottomright)
plot!(plt, t_arr[1:length] , v_excite[1:length], label = "V_Excite", color=col_exc)
plot!(plt, t_arr[1:length] , v_inhibit[1:length], label = "V_Inhibit", color=col_inh)

In [ ]:
length = size(t_arr,1)
plt = plot(xlabel="time (s)", ylabel="Rate (Hz)", fmt = :png, legend=:bottomright)
plot!(plt, t_arr[1:length] , rate[1:length,1], label = "r_Excite", color=col_exc)
plot!(plt, t_arr[1:length] , rate[1:length,2], label = "r_Inhibit", color=col_inh)

Running the SSN Simulation for t_max time, across different external input values h, with no external noise

In [ ]:
function run_ssn_multiple_h_no_noise!(num_h, h_arr, rates_ssn, v_ssn, v_rest)
    t_max = 1
    num_steps = Int(ceil(t_max/time_step))
    t_arr = zeros(num_steps)
    v_excite = zeros(num_steps)
    v_inhibit = zeros(num_steps)
    rate = zeros(num_steps, 2)
    cov_noise = [0.0 0.0
                0.0 0.0]
    @showprogress 1.0 "Running SSN multiple h..." for i in 1:num_h
        spike_trainE = Vector{Float64}()
        spike_trainI = Vector{Float64}()
        simulate_ssn!(h_arr[i], num_steps, t_arr, v_excite, v_inhibit, v_rest, rate, spike_trainE, spike_trainI, cov_noise)
        rates_ssn[i,:] = rate[end,:]
        v_ssn[i,1] = v_excite[end]
        v_ssn[i,2] = v_inhibit[end]
    end
end

In [ ]:
h_max = 20.0
h_min = 1.0
step = 1
num_h = Int(ceil((h_max-h_min+1)/step))
h_arr = zeros(num_h)
h_arr[1] = h_min
rates_ssn = zeros(num_h,2)
v_ssn = zeros(num_h, 2)

for i in 2:num_h
    h_arr[i] = h_arr[i-1] + step
end
run_ssn_multiple_h_no_noise!(num_h, h_arr, rates_ssn, v_ssn, v_rest)

In [ ]:
length = size(h_arr,1)
plt = plot(xlabel="time (s)", ylabel="Voltage (mV)", fmt = :png, legend=:bottomright)
plot!(plt, h_arr[1:length] , v_ssn[1:length, 1], label = "V_Excite", color=col_exc)
plot!(plt, h_arr[1:length] , v_ssn[1:length, 2], label = "V_Inhibit", color=col_inh)

In [ ]:
length = size(h_arr,1)
plt = plot(xlabel="time (s)", ylabel="Rate (Hz)", fmt = :png, legend=:bottomright)
plot!(plt, h_arr[1:length] , rates_ssn[1:length, 1], label = "r_Excite", color=col_exc)
plot!(plt, h_arr[1:length] , rates_ssn[1:length, 2], label = "r_Inhibit", color=col_inh)

Adding uncorrelated noise

In [ ]:
function single_h_noise_ssn_simulation()
    h = 5.0
    t_max = 100
    num_steps = Int(ceil(t_max/time_step))
    t_arr = zeros(num_steps)
    v_excite = zeros(num_steps)
    v_inhibit = zeros(num_steps)
    spike_trainE = Vector{Float64}()
    spike_trainI = Vector{Float64}()
    rate = zeros(num_steps, 2)
    cov_noise = [alpha*h 0.0
                0.0 alpha*h]
    simulate_ssn!(h, num_steps, t_arr, v_excite, v_inhibit, v_rest, rate, spike_trainE, spike_trainI, cov_noise)
    return t_arr, v_excite, v_inhibit, rate, spike_trainE, spike_trainI
end

In [ ]:
t_arr, v_excite, v_inhibit, rate, spike_trainE, spike_trainI = single_h_noise_ssn_simulation()
println([size(spike_trainE), size(spike_trainI)])

In [ ]:
v_e_mean = mean(v_excite)
v_i_mean = mean(v_inhibit)
r_e_mean = mean(rate[:,1])
r_i_mean = mean(rate[:,2])
println([v_e_mean, v_i_mean])
println([r_e_mean, r_i_mean])

In [ ]:
length = 5000
v_mean = zeros(length,2)
for i in 1:length
    v_mean[i,1] = v_e_mean
    v_mean[i,2] = v_i_mean
end
plt = plot(xlabel="time (s)", ylabel="Voltage (mV)", fmt = :png, legend=:bottomright)
plot!(plt, t_arr[1:length] , v_excite[1:length], label = "V_Excite", color=col_exc)
plot!(plt, t_arr[1:length] , v_inhibit[1:length], label = "V_Inhibit", color=col_inh)
plot!(plt, t_arr[1:length], v_mean, label = ["E[V_Excite]" "E[V_Inhibit]"], linestyle=:dash,linewidth=2, color=[col_exc col_inh])

In [ ]:
length = 5000
r_mean = zeros(length,2)
for i in 1:length
    r_mean[i,1] = r_e_mean
    r_mean[i,2] = r_i_mean
end
plt = plot(xlabel="time (s)", ylabel="Rate (Hz)", fmt = :png, legend=:topright)
plot!(plt, t_arr[1:length] , rate[1:length,1], label = "r_Excite", color=col_exc)
plot!(plt, t_arr[1:length] , rate[1:length,2], label = "r_Inhibit", color=col_inh)
plot!(plt, t_arr[1:length], r_mean, label = ["E[r_Excite]" "E[r_Inhibit]"], linestyle=:dash,linewidth=2, color=[col_exc col_inh])

In [ ]:
function plot_count(points)
    y = collect(1:length(points))
    plt = plot(xlabel="time (s)", ylabel="count")
    plot!(plt, points, y, label = "N(t)")
end

function plot_count(points_E, points_I)
    y = collect(1:length(points_E))
    plt = plot(xlabel="time (s)", ylabel="count")
    plot!(plt, points_E, y, label = "N_E(t)")
    y = collect(1:length(points_I))
    plot!(plt, points_I, y, label = "N_I(t)")
end

function plot_count(points_E, points_I, remean, rimean, count)
    y = collect(1:count)
    y1 = zeros(count)
    y2 = zeros(count)
    for i in 1:count
        y1[i] = remean*points_E[i]
        y2[i] = rimean*points_I[i]
    end
    plt = plot(xlabel="time (s)", ylabel="Spike Count", legend=:bottomright, fmt=:png)
    plot!(plt, points_E[1:count], y, label = "N_E(t)", color="blue")
    plot!(plt, points_I[1:count], y, label = "N_I(t)", color="red")
    plot!(plt, points_E[1:count],y1, label="E[N_E(t)]", color="dark blue")
    plot!(plt, points_I[1:count],y2, label="E[N_I(t)]", color="dark red")
end

In [ ]:
plot_count(spike_trainE, spike_trainI, r_e_mean, r_i_mean, 900)

In [ ]:
function rasterplot( spike_trainE, spike_trainI,tlims = (80.,90.))
  _trainE = spike_trainE
  plt=plot()
  trainE = filter(t-> tlims[1]< t < tlims[2],_trainE)
  nspk = size(trainE,1)
  scatter!(plt,trainE,fill(2,nspk),markersize=35, markercolor=:blue,markershape=:vline,legend=:topright, label="Excitatory")
  _trainI = spike_trainI
    trainI = filter(t-> tlims[1]< t < tlims[2],_trainI)
  nspk = size(trainI,1)
  scatter!(plt,trainI,fill(1,nspk),markersize=35, markercolor=:red,markershape=:vline,legend=:topright, label="Inhibitory")
  plot!(plt,ylims=(0,3),xlabel="time (s)",fmt=:png)
end

In [ ]:
rasterplot(spike_trainE, spike_trainI)